In [1]:
import pandas as pd
import useful_functions as use
import pyodbc

In [2]:
username = 'vigrose'
password = 'Ravenclaw~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password)
AMAEDW = pyodbc.connect(w)

In [5]:
ME_QUERY = \
        f"""
        SELECT DISTINCT
        P.PARTY_ID,
        P.KEY_VAL AS ME
        FROM
        AMAEDW.PARTY_KEY P
        WHERE
        P.KEY_TYPE_ID = 18
        AND
        P.ACTIVE_IND = 'Y'
        """
MES = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [3]:
sql_query = \
    f'''
    SELECT DISTINCT
    N.AREA_CD,
    N.EXCHANGE,
    N.PHONE_NBR,
    P.PARTY_ID,
    CC.DESC AS CATEGORY_DESC,
    C.PURPOSE_TYPE_DESC,
    C.PURPOSE_USG_DESC
    FROM
    AMAEDW.PHONE_NBR N, AMAEDW.PARTY_PHONE P, AMAEDW.CAT_CD CC, AMAEDW.CONT_PURPOSE_TYPE C
    WHERE
    P.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    N.PHONE_ID = P.PHONE_ID
    AND
    P.SRC_END_DT IS NULL
    AND
    CC.CAT_CD_ID = P.CAT_CD_ID
    AND
    C.PURPOSE_USG_CD = 'PV'
    AND
    C.PURPOSE_CAT_CD='P';
    '''
phone_source = pd.read_sql(con=AMAEDW, sql=sql_query)

In [6]:
phone_source = pd.merge(phone_source, MES, on='PARTY_ID')

In [7]:
npi_match = pd.read_csv('../../Data/NPI/npi_phone_match.csv')

In [18]:
NUMS = []
for row in phone_source.itertuples():
    NUM = f'{row.AREA_CD.strip()}{row.EXCHANGE.strip()}{row.PHONE_NBR.strip()}'
    NUMS.append(NUM)
phone_source['TELEPHONE_NUMBER']=NUMS

In [21]:
def fix_phone(num):
    num = str(num).strip().replace('.0', '')
    if num[0] == '1':
        num = num[1:]
    num = ''.join(filter(str.isdigit, num))
    num = num[:10]
    return num

In [22]:
npi_match['TELEPHONE_NUMBER']=[fix_phone(x) for x in npi_match.TELEPHONE_NUMBER]

In [11]:
npi_match['ME']=use.fix_me(npi_match.ME)

In [29]:
pd.merge(phone_source, npi_match, on=['ME','TELEPHONE_NUMBER']).groupby('CATEGORY_DESC').count()[['AREA_CD']]

,AREA_CD
CATEGORY_DESC,
Account Management Customer Communication,1602
CME Registration,32
Data Append Services,2660
EIB Communication for batch only,38732
External E-mail,1289
Federation,675
From Internet Search,246
Group Roster/Census,22853
HSG VHCP DATA,4990


In [32]:
humach = pd.read_csv('../../Data/Humach/humach_good_phones_since_180706.csv')

In [35]:
humach['ME'] = use.fix_me(humach.PHYSICIAN_ME_NUMBER)

In [39]:
humach['TELEPHONE_NUMBER']=[fix_phone(x) for x in humach.OFFICE_TELEPHONE]

In [45]:
humach['DATE'] = pd.to_datetime(humach.WSLIVE_FILE_DT)

In [46]:
VERFIED = pd.merge(humach, npi_match, on=['ME','TELEPHONE_NUMBER'])

In [48]:
verified = VERFIED.sort_values('DATE').drop_duplicates('ME', keep='last')

,PHYSICIAN_ME_NUMBER,PHYSICIAN_FIRST_NAME,PHYSICIAN_MIDDLE_NAME,PHYSICIAN_LAST_NAME,SUFFIX,DEGREE,OFFICE_ADDRESS_LINE_1,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,...,WSLIVE_FILE_DT,MATCH_ADDR,MATCH_PHONE,MATCH_ADDR_LONG,SPECIALTY,SPECIALTY_UPDATED,SOURCE_FILE_DT,ME,TELEPHONE_NUMBER,DATE
76053,4812841772,PAUL,B,TAYLOR,NaN,NaN,NaN,7150 GREENVILLE AVE STE 600,DALLAS,TX,...,07/13/2018,0481284177715075231,4812841772143283566,NaN,FM,1,NaN,04812841772,2143283566,2018-07-13
5121,514000557,DANIEL,S,BEHROOZAN,NaN,NaN,NaN,2221 LINCOLN BLVD # 100,SANTA MONICA,CA,...,07/13/2018,0051400055222190405,514000553103921111,NaN,D,1,NaN,00514000557,3103921111,2018-07-13
54543,3575031888,DIANA,LUISA,PAGE,NaN,NaN,STE G01,624 MCCLELLAN ST,SCHENECTADY,NY,...,07/13/2018,035750318862412304,3575031885183475655,NaN,FM,1,NaN,03575031888,5183475655,2018-07-13
66412,4113841341,MINH,QUANG,NGUYEN,NaN,NaN,NaN,2300 HIGHLAND AVE,BETHLEHEM,PA,...,07/13/2018,0411384134230018020,4113841346108618080,NaN,CD,1,NaN,04113841341,6108618080,2018-07-13
49830,3509071909,JOSHUA,ISAAC,MOZES,NaN,NaN,NaN,287 W JEFFERSON ST,BOISE,ID,...,07/13/2018,035090719028783702,3509071902083221680,NaN,NaN,0,NaN,03509071909,2083221680,2018-07-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37350,2604121178,ANDREW,JOHN,BRYAN,NaN,NaN,NaN,1515 NW 18TH AVE STE 300,PORTLAND,OR,...,07/01/2021,0260412117151597209,02604121175032248399,NaN,ORS,1,06/28/2021,02604121178,5032248399,2021-07-01
106635,74801680317,ALEJANDRO,A,EISMA,NaN,NaN,NaN,PO BOX 9,WINNEBAGO,WI,...,07/01/2021,7480168031POBOX954985,74801680319202354910,NaN,IM,1,06/28/2021,74801680317,9202354910,2021-07-01
40972,2846100639,AMIRA,NaN,GHAZALI,NaN,NaN,407 BURKARTH RD,SUIITE 302,WARRENSBURG,MO,...,07/01/2021,0284610063SUIITE64093,02846100636607475558,NaN,GS,1,06/28/2021,02846100639,6607475558,2021-07-01
10421,1002961021,BASIL,RASHAD,BESH,NaN,NaN,NaN,39180 FARWELL DR,FREMONT,CA,...,07/01/2021,01002961023918094538,01002961025108571000,NaN,ORS,1,06/28/2021,01002961021,5108571000,2021-07-01


In [42]:
len(npi_match)

344073